<a href="https://colab.research.google.com/github/panthamramesh/Qualcomm-DL-Hackathon/blob/master/GenAI_hacathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install groq
!pip install requests

In [ ]:
import os
from groq import Groq
os.environ["GROQ_API_KEY"] = "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"

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Test the client with a simple request
response = client.llm.list_models()
print(response)

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [ ]:
train_data = pd.read_csv('/content/train_5v5GIB2.csv')

In [ ]:
# Combine subject and body for input
train_data['text'] = train_data['ticket_subject'] + " " + train_data['ticket_body']

In [ ]:
# Define the system prompt
system_prompt = """
You are an AI model trained to classify IT support tickets. Each ticket consists of a subject and body, and you need to predict the following attributes:
- Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
- Priority: The urgency level such as low, medium, or high.
- Language: The language code for the email's language, such as en (English), de (German) etc.
- Type: The nature of the request, categorized as Incident, Request, Problem, or Change.
"""

# Craft few-shot prompts for training examples
def create_few_shot_prompt(train_data):
    examples = "\n\n".join([f"Subject: {row['ticket_subject']}\nBody: {row['ticket_body']}\n\nClassify the following attributes:\nDepartment: {row['department']}\nPriority: {row['priority']}\nLanguage: {row['language']}\nType: {row['type']}" for _, row in train_data.iterrows()])
    return examples

few_shot_prompt = create_few_shot_prompt(train_data)


In [ ]:
few_shot_prompt

"Subject: Discrepancia de facturación en Google Workspace\nBody: Estimado equipo de soporte de TI,\n\nEstoy escribiendo para informar un monto de facturación incorrecto en mi suscripción de Google Workspace Business Standard bajo la cuenta <acc_num>. Por favor, revise y ajuste la factura. Espero su pronta respuesta.\n\nSaludos,\n\n<name>\n\nClassify the following attributes:\nDepartment: Billing and Payments\nPriority: low\nLanguage: es\nType: Incident\n\nSubject: Urgent Consultation Request for Critical IT Issues\nBody: Dear IT Services Support Team, I hope this message finds you well. My name is <name> and I am currently experiencing critical issues with our server administration, which are significantly impacting our operations. We rely heavily on your IT Consulting Service for our ongoing technical needs. Therefore, we urgently need your expert assistance to resolve these issues as quickly as possible. Time is of the essence because our operations have come to a standstill. Please 

In [ ]:
# Prepare training data for Groqcloud
train_prompts = [few_shot_prompt] * len(train_data)
train_labels = train_data[['type', 'department', 'language', 'priority']].values.tolist()